### Import package

In [55]:
import numpy as np
import matplotlib.pyplot as plt
import flopy
import flopy.mf6 as mf6
from flopy.discretization import StructuredGrid
from flopy.utils import Raster
from osgeo import gdal


### Setup model

In [56]:
model_name = 'peat_testing_3'
workspace = "../testing_peat_module/Model_Testing_3"

### Using flopy's raster class for discretization

In [ ]:
# modelgrid = StructuredGrid(
#     delr=np.ones(ncol) * delr,
#     delc=np.ones(nrow) * delc,
#     xoff=xoffset, yoff=yoffset, angrot=0)

# br_surf = Raster.load("../testing_peat_module/Raster/DEMNAS_1012-53_v1.0_48s_fill_100m_clip.tif")
# rs = br_surf.resample_to_grid(
#     modelgrid, band=1, method='linear')
# np.savetxt('../testing_peat_module/Model_Testing_3/data/top.dat', rs)

# bottom_1 = Raster.load("../testing_peat_module/Raster/peatdepth_0_3_mtr.tif")
# bt_1 = bottom_1.resample_to_grid(
#     modelgrid, band=1, method='linear')
# np.savetxt('../testing_peat_module/Model_Testing_3/data/bottom_1.dat', bt_1)

# # top and bottom grid
# top = [{'filename': 'data/top.dat'}]
# # botm = [{'filename': 'data/bottom_1.dat'}]

# # hydraulic conductivity

# # use the model top for starting heads
# strt = [top[0]] * nlay

# dis = mf6.ModflowGwfdis(
#     gwf, nlay=nlay, nrow=nrow, ncol=ncol,
#     delr=delr, delc=delc, 
#     top=top, botm=0, idomain=1)

# locate the model grid
# gwf.modelgrid.set_coord_info(
#     xoff=xoffset, yoff=yoffset, epsg=epsg)
# gwf.modelgrid

### Discretization using numpy

In [57]:
# Raster paths
demPath = "../testing_peat_module/Raster/INA_1sec_srtm_refine_48s_100m_clip.tif"
# Open files
demDs = gdal.Open(demPath)
# Get data as arrays
demData = demDs.GetRasterBand(1).ReadAsArray()

### Setup MODFLOW model

In [58]:
sim = mf6.MFSimulation(
    sim_name=model_name, version="mf6", 
    exe_name="mf6", sim_ws=workspace)

### Setup temporal discretization and solver package

In [59]:
tdis = mf6.ModflowTdis(
    sim, time_units="days", nper=1, 
    perioddata=[(1.0, 1, 1.0)])

#perioddata: stress period, timestep in a stress period, multiplier

ims = mf6.ModflowIms(
    sim, complexity="moderate", 
    outer_dvclose=0.001)

### Setup spatial discretization

In [60]:
# create the model instance
gwf = mf6.ModflowGwf(
    sim, modelname=model_name, 
    save_flows=True)

# output control
oc = mf6.ModflowGwfoc(
    gwf, head_filerecord=f'{gwf.name}.hds', 
    budget_filerecord=f'{gwf.name}.cbc',
    saverecord=[('head', 'all'), ("budget", "all")])

# set up the discretization package
Nrow = demDs.RasterXSize
Ncol = demDs.RasterYSize
cell_size = demDs.GetGeoTransform()[1]
ztop = demDs.ReadAsArray()
zbot = np.linspace(-10/1, -10, 1 ) # set constant thickness of 10 units

dis = flopy.mf6.ModflowGwfdis(gwf, nlay=1, nrow=Nrow, ncol=Ncol, delr=cell_size, delc=cell_size,
                              top=ztop, botm=zbot)


zbot

array([-10.])

### Assign aquifer properties
Since this is a steady-state model, we need only include the Node Property Flow (NPF) package, which specifies hydraulic conductivity.

In [61]:
hk = [1E-6] # set hydraulic conductivity value uniformly across the model
npf = mf6.ModflowGwfnpf(
        gwf, pname="npf", icelltype=1, k=hk, save_flows=True
)

### Assign initial condition

In [62]:
strt = 3 #meter
# or can be adapted from external initial head source
ic = mf6.ModflowGwfic(gwf, strt=strt)

### Assign boundary condition (not yet assigned)

#### recharge

In [54]:
index = np.zeros((Nrow, Ncol), dtype=float)
#apply recharge on selected cells
rchr = 0.2/365/86400
rchr_list = []
for i in index:
    rchr_list.append([0,i,rchr])
rchr_spd = {0:rchr_list}
#recharge package
rch = flopy.mf6.ModflowGwfrch(gwf, stress_period_data=rchr_spd)


#### constant head boundary

In [ ]:
# constant head boundary
# chd = mf6.ModflowGwfchd #(gwf, stress_period_data=spec_head_perimeter)

#### drainage

In [ ]:
# drainage

#### river

### Write simulation file

In [63]:
sim.write_simulation()

writing simulation...
  writing simulation name file...
  writing simulation tdis package...
  writing ims package ims_-1...
  writing model peat_testing_3...
    writing model name file...
    writing package oc...
    writing package dis...
    writing package npf...
    writing package ic...


### Run the model

In [64]:
sim.run_simulation()

FloPy is using the following executable to run the model: C:\Users\mwidijanto\AppData\Local\flopy\bin\mf6.EXE
                                   MODFLOW 6
                U.S. GEOLOGICAL SURVEY MODULAR HYDROLOGIC MODEL
                        VERSION 6.4.1 Release 12/09/2022

   MODFLOW 6 compiled Dec 10 2022 05:57:01 with Intel(R) Fortran Intel(R) 64
   Compiler Classic for applications running on Intel(R) 64, Version 2021.7.0
                             Build 20220726_000000

This software has been approved for release by the U.S. Geological 
Survey (USGS). Although the software has been subjected to rigorous 
review, the USGS reserves the right to update the software as needed 
pursuant to further analysis and review. No warranty, expressed or 
implied, is made by the USGS or the U.S. Government as to the 
functionality of the software and related material nor shall the 
fact of release constitute any such warranty. Furthermore, the 
software is released on condition that neither t

(True, [])

### Post procesing

In [ ]:
from flopy.utils.postprocessing import get_water_table

hds = gwf.output.head().get_data()
wt = get_water_table(hds)

In [ ]:
cbc = gwf.output.budget()

In [ ]:
fig, ax = plt.subplots(figsize=(6, 6))
pmv = flopy.plot.PlotMapView(gwf, ax=ax)
ctr = pmv.contour_array(
    wt, levels=np.arange(290, 315, 1), 
    linewidths=1, colors='b')
labels = pmv.ax.clabel(
    ctr, inline=True, 
    fontsize=8, inline_spacing=1)
vmin, vmax = -100, 100
# im = pmv.plot_array(
#     lak[0], cmap='coolwarm', 
#     vmin=vmin, vmax=vmax)
# im = pmv.plot_array(
#     sfr.sum(axis=0), cmap='coolwarm', 
#     vmin=vmin, vmax=vmax)
# cb = fig.colorbar(
#     im, shrink=0.5, label='Leakage, in m$^3$/day')
# ax.set_ylabel("Northing, WTM meters")
# ax.set_xlabel("Easting, WTM meters")
ax.set_aspect(1)
plt.tight_layout()
# plt.savefig('results.pdf')